In [18]:
import json
import pandas as pd
import numpy as np
from utils.data_helper import get_markable_dataframe, get_embedding_variables
from functools import reduce
from IPython.display import HTML, display
from tabulate import tabulate
from html import escape

In [19]:
embedding_indexes_file_path = 'helper_files/embedding/embedding_indexes.txt'
indexed_embedding_file_path = 'helper_files/embedding/indexed_embedding.txt'

word_vector, embedding_matrix, idx_by_word, word_by_idx = get_embedding_variables(embedding_indexes_file_path, indexed_embedding_file_path)

In [20]:
markables_original_singletons = get_markable_dataframe("data/testing/markables.csv", word_vector, idx_by_word)
markables_predicted_singletons = get_markable_dataframe("data/testing/markables_with_predicted_singleton.csv", word_vector, idx_by_word)

markables_original_singletons.head()

id                      text  is_pronoun                     entity_type  \
0  1916  [1263, 1264, 1968, 1395]           0  [0, 0, 0, 0, 1, 0, 1, 0, 0, 0]   
1  1917                     [213]           1  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]   
2  1918        [1263, 1969, 1188]           0  [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]   
3  1919          [1470, 25, 1161]           0  [0, 0, 0, 0, 0, 1, 1, 0, 0, 0]   
4  1920                     [424]           0  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0]   

   is_proper_name  is_first_person  \
0               1                0   
1               0                0   
2               1                0   
3               0                0   
4               0                0   

                                      previous_words  \
0                                                 []   
1           [1263, 1264, 1968, 1395, 999, 379, 1161]   
2  [1263, 1264, 1968, 1395, 999, 379, 1161, 213, 27]   
3  [1968, 1395, 999, 379, 1161, 213, 27, 1263, 19...   
4  [1161, 213, 27, 1263, 1969, 1188, 1470, 25, 11...   

                                          next_words is_singleton  
0  [999, 379, 1161, 213, 27, 1263, 1969, 1188, 14...   [1.0, 0.0]  
1  [27, 1263, 1969, 1188, 1470, 25, 1161, 63, 424...   [1.0, 0.0]  
2  [1470, 25, 1161, 63, 424, 1223, 25, 1415, 1161...   [1.0, 0.0]  
3  [63, 424, 1223, 25, 1415, 1161, 876, 344, 213,...   [0.0, 1.0]  
4  [1223, 25, 1415, 1161, 876, 344, 213, 406, 122...   [0.0, 1.0]

In [38]:
markables_predicted_singletons[markables_predicted_singletons['id'].map(lambda x: x in [2655, 2677, 2692, 2693, 2697])]

id                      text  is_pronoun  \
469  2655                    [2320]           0   
491  2677                    [2320]           0   
506  2692  [1263, 2341, 1494, 2342]           0   
507  2693                    [2320]           0   
511  2697                    [2320]           0   

                        entity_type  is_proper_name  is_first_person  \
469  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]               1                0   
491  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]               1                0   
506  [0, 0, 0, 0, 1, 1, 0, 0, 0, 0]               1                0   
507  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]               1                0   
511  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]               1                0   

                                        previous_words  \
469  [2316, 1211, 1707, 2317, 2150, 2, 1263, 2318, ...   
491  [281, 1034, 1360, 14, 1015, 243, 2237, 2329, 8...   
506  [851, 2336, 516, 2337, 2319, 2338, 791, 2339, ...   
507  [2319, 2338, 791, 2339, 2340, 30, 1263, 2341, ...   
511  [2148, 334, 1263, 2318, 2337, 2319, 1249, 227,...   

                                            next_words is_singleton  
469  [30, 833, 2321, 1218, 1662, 25, 1476, 1034, 25...   [0.0, 1.0]  
491  [197, 256, 851, 14, 2330, 27, 1242, 1494, 2331...   [0.0, 1.0]  
506  [2320, 263, 213, 2148, 334, 1263, 2318, 2337, ...   [0.0, 1.0]  
507  [263, 213, 2148, 334, 1263, 2318, 2337, 2319, ...   [1.0, 0.0]  
511  [2268, 544, 218, 1263, 2318, 2077, 2319, 330, ...   [1.0, 0.0]

In [21]:
def get_markable_text(idx):
    return ' '.join([word_by_idx[x] for x in markables_original_singletons[markables_original_singletons['id'] == idx].text.values[0]])

def get_markable_previous_words(idx):
    return ' '.join([word_by_idx[x] for x in markables_original_singletons[markables_original_singletons['id'] == idx].previous_words.values[0]])

def get_markable_next_words(idx):
    return ' '.join([word_by_idx[x] for x in markables_original_singletons[markables_original_singletons['id'] == idx].next_words.values[0]])

def get_markable(idx):
    return f'{get_markable_previous_words(idx)} [{get_markable_text(idx).upper()}] {get_markable_next_words(idx)}'.strip()

In [22]:
right_predicted_markables = markables_predicted_singletons[markables_predicted_singletons['is_singleton'].map(lambda x: x[0]) == markables_original_singletons['is_singleton'].map(lambda x: x[0])]
wrong_predicted_markables = markables_predicted_singletons[markables_predicted_singletons['is_singleton'].map(lambda x: x[0]) != markables_original_singletons['is_singleton'].map(lambda x: x[0])]

# Analysis of Wrongly Predicted Markables

## Label: singleton, predicted: non-singleton

In [23]:
false_negatives = wrong_predicted_markables[wrong_predicted_markables['is_singleton'].map(lambda x: x[0]) == 1]
false_negative_texts = [get_markable(idx) for idx in false_negatives['id']]

In [24]:
data_to_analyze =  false_negatives

print(f'"nya": {len(list(filter(lambda x: get_markable_text(x) == "nya", data_to_analyze["id"])))}')
print(f'proper name: {len(data_to_analyze[data_to_analyze["is_proper_name"] == 1])}')
print(f'pronoun: {len(data_to_analyze[data_to_analyze["is_pronoun"] == 1])}')   
print(f'pronoun without "nya": {len(data_to_analyze[data_to_analyze["is_pronoun"] == 1]) - len(list(filter(lambda x: get_markable_text(x) == "nya", data_to_analyze["id"])))}')
print(f'first person: {len(data_to_analyze[data_to_analyze["is_first_person"] == 1])}')

"nya": 5
proper name: 19
pronoun: 5
pronoun without "nya": 0
first person: 0


In [25]:
data_to_analyze, text_data_to_analyze = false_negatives, false_negative_texts

display(HTML(tabulate([
    [i, idx, escape(text)]
    for i, idx, text
    in zip(range(1, len(data_to_analyze) + 1), data_to_analyze['id'], text_data_to_analyze)
], tablefmt='html')))

## Label: non-singleton, predicted: singleton

In [26]:
false_positives = wrong_predicted_markables[wrong_predicted_markables['is_singleton'].map(lambda x: x[0]) == 0]
false_positive_texts = [get_markable(idx) for idx in false_positives['id']]

In [27]:
data_to_analyze =  false_positives

print(f'"nya": {len(list(filter(lambda x: get_markable_text(x) == "nya", data_to_analyze["id"])))}')
print(f'proper name: {len(data_to_analyze[data_to_analyze["is_proper_name"] == 1])}')
print(f'pronoun: {len(data_to_analyze[data_to_analyze["is_pronoun"] == 1])}')   
print(f'pronoun without "nya": {len(data_to_analyze[data_to_analyze["is_pronoun"] == 1]) - len(list(filter(lambda x: get_markable_text(x) == "nya", data_to_analyze["id"])))}')
print(f'first person: {len(data_to_analyze[data_to_analyze["is_first_person"] == 1])}')

"nya": 7
proper name: 22
pronoun: 9
pronoun without "nya": 2
first person: 0


In [28]:
data_to_analyze, text_data_to_analyze = false_positives, false_positive_texts

display(HTML(tabulate([
    [i, idx, escape(text)]
    for i, idx, text
    in zip(range(1, len(data_to_analyze) + 1), data_to_analyze['id'], text_data_to_analyze)
], tablefmt='html')))

# Analysis of Rightly Predicted Markables

## Non-Singletons

In [29]:
true_negatives = right_predicted_markables[right_predicted_markables['is_singleton'].map(lambda x: x[0]) == 1]
true_negative_texts = [get_markable(idx) for idx in true_negatives['id']]

In [30]:
data_to_analyze =  true_negatives

print(f'"nya": {len(list(filter(lambda x: get_markable_text(x) == "nya", data_to_analyze["id"])))}')
print(f'proper name: {len(data_to_analyze[data_to_analyze["is_proper_name"] == 1])}')
print(f'pronoun: {len(data_to_analyze[data_to_analyze["is_pronoun"] == 1])}')   
print(f'pronoun without "nya": {len(data_to_analyze[data_to_analyze["is_pronoun"] == 1]) - len(list(filter(lambda x: get_markable_text(x) == "nya", data_to_analyze["id"])))}')
print(f'first person: {len(data_to_analyze[data_to_analyze["is_first_person"] == 1])}')

"nya": 19
proper name: 41
pronoun: 30
pronoun without "nya": 11
first person: 0


In [31]:
data_to_analyze, text_data_to_analyze = true_negatives, true_negative_texts

display(HTML(tabulate([
    [i, idx, escape(text)]
    for i, idx, text
    in zip(range(1, len(data_to_analyze) + 1), data_to_analyze['id'], text_data_to_analyze)
], tablefmt='html')))

## Singletons

In [32]:
true_positives = right_predicted_markables[right_predicted_markables['is_singleton'].map(lambda x: x[0]) == 0]
true_positive_texts = [get_markable(idx) for idx in true_positives['id']]

In [33]:
data_to_analyze =  true_positives

print(f'"nya": {len(list(filter(lambda x: get_markable_text(x) == "nya", data_to_analyze["id"])))}')
print(f'proper name: {len(data_to_analyze[data_to_analyze["is_proper_name"] == 1])}')
print(f'pronoun: {len(data_to_analyze[data_to_analyze["is_pronoun"] == 1])}')   
print(f'pronoun without "nya": {len(data_to_analyze[data_to_analyze["is_pronoun"] == 1]) - len(list(filter(lambda x: get_markable_text(x) == "nya", data_to_analyze["id"])))}')
print(f'first person: {len(data_to_analyze[data_to_analyze["is_first_person"] == 1])}')

"nya": 18
proper name: 92
pronoun: 25
pronoun without "nya": 7
first person: 0


In [34]:
data_to_analyze, text_data_to_analyze = true_positives, true_positive_texts

display(HTML(tabulate([
    [i, idx, escape(text)]
    for i, idx, text
    in zip(range(1, len(data_to_analyze) + 1), data_to_analyze['id'], text_data_to_analyze)
], tablefmt='html')))